# Week 3: Segmenting and Clustering Neighborhoods in Toronto



### Scraping data from wikipedia page

I decided to use both pandas and beautifulsoup to scrape data from wikipedia page. I'm using beautifulsoup to find table with postal codes, boroughs and neighbourhoods, then usind pandas read_html to get data from html table

In [1]:
#importing libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#getting page content with BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

#finding table with data
table=soup.find_all('table', class_='wikitable sortable')

#read data from html with pandas
df_tmp = pd.read_html(str(table))[0]


There are no Boroughs with **Not assigned** neighborhoods

In [2]:
df_tmp[df_tmp['Neighborhood'] == 'Not assigned'].count()

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64

### Cleaning data and grouping neighborhoods

In [3]:
#dropping rows containg NaN
df_tmp.dropna(inplace=True)

#Grouping neighborhoods with same postal code
df = df_tmp.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [4]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df.shape

(103, 3)

In [6]:
coords=pd.read_csv("http://cocl.us/Geospatial_data")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#make a copy of data frame
df1=df.copy()

#Merge it with Geo data to get Latitude and Longtitude
dfr=df1.merge(coords, how='left')
dfr.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
dfr.shape

(103, 5)